In [ ]:
pip install modin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 956 kB 5.2 MB/s 
     |████████████████████████████████| 956 kB 41.2 MB/s 
     |████████████████████████████████| 761 kB 31.3 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
path_1 = "/content/drive/MyDrive/Colab/BD_Tec_Banderas_2018_2021_3Estaciones.xlsx" 
path_2 = "/content/drive/MyDrive/Colab/BD_Tec_banderas_contaminantes.xlsx"
path_3 = "/content/drive/MyDrive/Colab/BD_Tec_Banderas_meteo.xlsx"


In [ ]:
import xlrd

from pandas.core import apply

# A esta función le das la ruta del archivo xlsx de varias hojas y te regresa un 
# diccionario con el nombre de cada hoja como llave la cual contiene el dataframe 
# de dicha hoja.

def read_all_sheets(path):  
  df_dict = {}
  for i in xlrd.open_workbook(path, on_demand=True).sheet_names(): 
    globals()[f'{i}'] = pd.read_excel(path, sheet_name=i)
    df_dict[i] = globals()[f'{i}']
  return df_dict

# A esta función le das un diccionario que guarde dataframes en cada uno de sus elementos, en este caso 
# para las banderas te lee cada una de las banderas existentes de todas las hojas y te regresa una lista 
# con todas las banderas existentes

def flags(df_dict):
  flags = set()
  for key in df_dict.keys():
    df = df_dict[key][key+' b'].value_counts().rename_axis('unique_values').reset_index(name='counts')
    flags = flags | set(df['unique_values'])
    flags_list = list(flags)
  return flags_list 


# A esta función le das un diccionario que guarde dataframes y la lista de banderas existentes
# Para que así te junte en un dataframe todas las banderas y te diga cuantas tiene cada variable

def merge_flags(df_dict, flags_list):
  df = pd.DataFrame({'Flags':flags_list})
  for key in df_dict.keys():  
    aux_df = df_dict[key][key+'f'].value_counts().rename_axis('Flags').reset_index(name=key)
    df = df.merge(aux_df,how='left', left_on='Flags', right_on='Flags').fillna(0)
  df[list(df_dict.keys())] = df[list(df_dict.keys())].astype(int)
  return df

#A esta función le das el diccionario que guarde dataframes y te regresa todos los dataframes existentes
# en un sólo dataframe tomando siempre el de mayor longitud al que se le va a agregar el otro para que 
# no se eliminen fechas y los que no tienen registros en dichas fechas se rellenen con NaN 

def merge_data_no_flags(df_dict, datetime_column ):
  df = pd.DataFrame({datetime_column: []})
  for key in df_dict.keys():
    min = df.shape[0]
    if df_dict[key].shape[0] > min:
      df = df_dict[key].drop(key+'f', axis=1).merge(df,how='left', left_on=datetime_column, right_on=datetime_column)
    else: 
      df = df.merge(df_dict[key].drop(key+'f', axis=1),how='left', left_on=datetime_column, right_on=datetime_column)
  return df

def merge_data_with_flags(df_dict, datetime_column):
  df = pd.DataFrame({datetime_column: []})
  for key in df_dict.keys():
    min = df.shape[0]
    if df_dict[key].shape[0] > min:
      df = df_dict[key].merge(df,how='left', left_on=datetime_column, right_on=datetime_column)
    else: 
      df = df.merge(df_dict[key],how='left', left_on=datetime_column, right_on=datetime_column)
  df = df.rename(columns={datetime_column:"date"})
  return df

# Esta función te mezcla dos diccionarios.

def merge_two_dicts(x, y):
    z = x.copy() 
    z.update(y)  
    return z

# A esta función le das un Dataframe y te regresa los estadísticos principales

def df_stats(data_stats): 
  data_stats = data_stats.select_dtypes(include=np.number)
 
  stats = pd.concat([
            data_stats.describe().T,
            data_stats.mode(numeric_only=True).iloc[0].rename("Mode"),
            data_stats.var( numeric_only=True).rename("Var"),
            data_stats.sem(numeric_only=True).rename('SE Mean'),
            data_stats.isnull().sum().rename('N*')
            ],
            axis = 1)
  stats = stats.rename(columns={"count": "N",
                            "std": "StdDev",
                            "25%":"Q1",
                            "75%":"Q3",
                            "50%":"Median",
                            "min":"Minimum",
                            "max":"Maximum",
                            "mean":"Mean"
                            })
  column_names = ["N",
                  "N*",
                  "Mean", 
                  "SE Mean", 
                  "StdDev", 
                  "Var", 
                  "Minimum", 
                  "Maximum", 
                  "Q1", 
                  "Median", 
                  "Q3", 
                  "Mode"]
  return stats.reindex(columns=column_names)


#A esta función le das un dataframe y te regresa un diccionario con varios dataframes divididos 
# entre los años que existen en las fechas
def data_by_years(df, datetime_column, data_type):
  df[datetime_column] = pd.to_datetime(df[datetime_column], format='%Y-%m-%d %H:%M:%s')
  min_year = min(df[datetime_column]).year
  max_year = max(df[datetime_column]).year
  years_dict = {}
  for i in range(min_year, max_year+1):
      
    x = df.loc[(df[datetime_column] >= str(i)+'-01-01')
                  & (df[datetime_column] < str(i+1)+'-01-01')]
    years_dict[data_type+'_'+str(i)] = x

  return years_dict

#A esta función le das un dataframe de solamente 1 año y te regresa un diccionario con varios dataframes divididos 
# entre los meses que existen en las fechas

def data_by_months(df, datetime_column): 
  df[datetime_column] = pd.to_datetime(df[datetime_column], format='%Y-%m-%d %H:%M:%s') 
  months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
  year = min(df[datetime_column]).year
  months_dict = {}
  for i in months:
    if i != '12':
      month = df.loc[(df[datetime_column] >= str(year)+'-'+i+'-01')
                    & (df[datetime_column] < str(year)+'-'+i+'-01')]
    else:
      month = df.loc[(df[datetime_column] >= str(year)+'-'+i+'-01')
                    & (df[datetime_column] < str(year+1)+'-01-01')]
    months_dict['data_'+str(year)+'_'+i] = month
  return months_dict 

def apply_flags(df, data_type):
  if data_type == 'meteo':
    for i in df.loc[df['SRf'] =='l'].index:
      df.SR[i] = np.nan
      df.SRf[i] = 0
    for i in df.loc[df['WDf'] =='D'].index:
      df.WDf[i] = df.WDf[i]   
    for key in list(df.select_dtypes(include='number').columns):
      df[key+'f'] = df[key+'f'].fillna(0)
      for i in df.loc[df[key+'f'] != 0].index:
        df[key][i] = np.nan 
      df = df.drop(key+'f', axis=1)
  if data_type == 'pollu':
    for key in list(df.select_dtypes(include='number').columns):
      df[key+'f'] = df[key+'f'].fillna(0)
      for i in df.loc[df[key+'f'] != 0].index:
        df[key][i] = np.nan
      df = df.drop(key+'f', axis=1)
  return df




def merge_dataframes(df1, df2, column_to_merge):
  if df2.shape[0] > df1.shape[0]:
    df = df2.merge(df1,how='left', left_on=column_to_merge, right_on=column_to_merge)
  else: 
    df = df1.merge(df2,how='left', left_on=column_to_merge, right_on=column_to_merge)
  return df


 # A esta función le das una ruta en la quieras guardar un dataframe, el dataframe que quieras guardar y el nombre
 # con el que lo quieras guardar y te lo guarda en DRIVE 
  
def save_to_Drive(path,df,name): 
  route = path+name+'.csv'
  with open(route, 'w', encoding = 'utf-8-sig') as f:
    df.to_csv(f, index=False)

def data_description(df): 
  description = pd.concat(
    [df.count().rename('N'),
     df.isnull().sum().rename('N*'),
    df.dtypes.rename("Data Type"),
    df.apply(pd.Series.nunique).rename("# uniq"),
    df.apply(pd.Series.unique).rename("Valores")
    ],
    axis = 1
  )

  description = description.reset_index()
  description = description.rename(columns={"index":"Abreviatura"})

  path = "/content/drive/Shareddrives/DatosSIMA/variables_meanings.xlsx"

  meaning = pd.read_excel(path)

  description = merge_dataframes(meaning, description, 'Abreviatura')
  return description

In [ ]:
df1 = read_all_sheets(path_1)
df2 = read_all_sheets(path_2)
df3 = read_all_sheets(path_3)

In [ ]:
for i in df1.keys():
  df1[i].date = df1[i].date.str[:-4]

In [ ]:
for i in df2.keys():
  df2[i] = df2[i].rename(columns={"Fecha":"date"})
for i in df3.keys():
  df3[i] = df3[i].rename(columns={"Fecha":"date"})

In [ ]:
df2.pop("LEEME")
df2.pop("Hoja1")

,Notas a considerar:,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,1. Información de los datos promedios diarios ...,NaN,NaN,NaN
2,2. La información se presenta dividida en libr...,NaN,NaN,NaN
3,3. A continuación se describe cada abreviatura...,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
56,e,Eliminar datos NO y Nox,Invalida,NaN
57,a,Eliminar PM menor a 5 ug/m3 y 0.05 ppm en CO,Invalida,NaN
58,s,Valores iguales consecutivos,Invalida,NaN
59,f,Valores 3 veces mayor que el valor anterior pa...,Invalida,NaN


In [ ]:
for i in df1.keys():
  df1[i] = df1[i][['date','Norte2', 'NTE2_b']]

In [ ]:
for i in df2.keys():
  df2[i] = df2[i][['date', 'SE', 'SE b', 'CE', 'CE b', 'SO2', 'SO2 b']]
for i in df3.keys():
  df3[i] = df3[i][['date', 'SE', 'SE b', 'CE', 'CE b', 'SO2', 'SO2 b']]
  

In [ ]:
df1['WS'] = df1.pop('WSR')
df1['WD'] = df1.pop('WDR')
df1.keys()

dict_keys(['PM10', 'PM2.5', 'O3', 'NO', 'NO2', 'NOx', 'SO2', 'CO', 'PRS', 'RH', 'TOUT', 'SR', 'RAINF', 'WS', 'WD'])

In [ ]:
df3.keys()

dict_keys(['TOUT', 'RH', 'SR', 'RAINF', 'PRS', 'WS', 'WD'])

In [ ]:
for i in df1.keys():
  h = i.lower()
  df1[i] = df1[i].rename(columns={'Norte2':h + '_NTE2', 'NTE2_b': h+'_NTE2f'})

In [ ]:
for i in df2.keys():
  h = i.lower()
  df2[i] = df2[i].rename(columns={'SE':h +'_SE', 'SE b':h + '_SEf', 'CE':h + '_CE', 'CE b':h + '_CEf', 'SO2':h + '_SO2', 'SO2 b':h + '_SO2f'})
for i in df3.keys():
  h = i.lower()
  df3[i] = df3[i].rename(columns={'SE':h +'_SE', 'SE b':h + '_SEf', 'CE':h + '_CE', 'CE b':h + '_CEf', 'SO2':h + '_SO2', 'SO2 b':h + '_SO2f'})

In [ ]:
data1 = merge_data_with_flags(df1, 'date')
data2 = merge_data_with_flags(df2, 'date')
data3 = merge_data_with_flags(df3, 'date')

In [ ]:
df1['WS']

,date,ws_NTE2,ws_NTE2f
0,2018-01-01 00:00:00,NaN,n
1,2018-01-01 01:00:00,15.5,
2,2018-01-01 02:00:00,15.3,
3,2018-01-01 03:00:00,13.0,
4,2018-01-01 04:00:00,13.3,
...,...,...,...
31837,2021-08-25 19:00:00,11.5,
31838,2021-08-25 20:00:00,9.6,
31839,2021-08-25 21:00:00,12.0,
31840,2021-08-25 22:00:00,11.4,


In [ ]:
data2['date'] = data2['date'].astype(str)
data3['date'] = data3['date'].astype(str)

In [ ]:
data12 = merge_dataframes(data1, data2,'date')
data = merge_dataframes(data12, data3,'date')

In [ ]:
data.columns

Index(['date', 'pm2.5_SE', 'pm2.5_SEf', 'pm2.5_CE', 'pm2.5_CEf', 'pm2.5_SO2',
       'pm2.5_SO2f', 'pm10_SE', 'pm10_SEf', 'pm10_CE',
       ...
       'ws_CE', 'ws_CEf', 'ws_SO2', 'ws_SO2f', 'wd_SE', 'wd_SEf', 'wd_CE',
       'wd_CEf', 'wd_SO2', 'wd_SO2f'],
      dtype='object', length=121)

In [ ]:
path = "/content/drive/MyDrive/Colab/"
save_to_Drive(path, data, "all_stations_data")

In [ ]:
data.pm

Index(['date', 'pm2.5_SE', 'pm2.5_SEf', 'pm2.5_CE', 'pm2.5_CEf', 'pm2.5_SO2',
       'pm2.5_SO2f', 'pm10_SE', 'pm10_SEf', 'pm10_CE',
       ...
       'ws_CE', 'ws_CEf', 'ws_SO2', 'ws_SO2f', 'wd_SE', 'wd_SEf', 'wd_CE',
       'wd_CEf', 'wd_SO2', 'wd_SO2f'],
      dtype='object', length=121)

In [ ]:
for i in df2.keys():
  print(i)
df3['RAINF'][['date', 'SE', 'SE b', 'CE', 'CE b', 'SO2', 'SO2 b']]


PM10
PM2.5
SO2
O3
CO
NO
NO2
NOx


,date,SE,SE b,CE,CE b,SO2,SO2 b
0,2017-01-01 00:00:00,0.00,NaN,0.0,NaN,0.00,NaN
1,2017-01-01 01:00:00,0.00,NaN,0.0,NaN,0.00,NaN
2,2017-01-01 02:00:00,0.00,NaN,0.0,NaN,0.00,NaN
3,2017-01-01 03:00:00,0.00,NaN,0.0,NaN,0.00,NaN
4,2017-01-01 04:00:00,0.00,NaN,0.0,NaN,0.00,NaN
...,...,...,...,...,...,...,...
39386,2021-06-30 19:00:00,0.00,NaN,0.0,NaN,0.05,NaN
39387,2021-06-30 20:00:00,0.04,NaN,0.0,NaN,0.00,NaN
39388,2021-06-30 21:00:00,0.06,NaN,0.0,NaN,0.00,NaN
39389,2021-06-30 22:00:00,0.02,NaN,0.0,NaN,0.00,NaN
